# PHNN for a damped mass-spring system

We train a [Hamiltonian neural network (HNN)](https://proceedings.neurips.cc/paper/2019/hash/26cd8ecadce0d4efd6cc8a8725cbd1f8-Abstract.html) (or [pseudo-Hamiltonian neural network (PHNN)](https://doi.org/10.1016/j.physd.2023.133673)) model to learn a (damped) mass-spring system.

#### Exercises:
* Change `TRAJ_TRAIN` to experiment with training on one or several trajectory, and add noise to the data
* Set up the neural network for the standard model and the PHNN by specifying the input and output dimensions
* Make the damping coefficient trainable in the HNN, so that it becomes a PHNN
* Change the damping coefficient `MU` to train on systems with and without damping

In [ ]:
try:
    import matplotlib.pyplot as plt
    import numpy as np
    import torch
    import torch.nn as nn
    from tqdm import trange
    import seaborn as sns
except ModuleNotFoundError:
    import subprocess
    import sys
    subprocess.check_call([sys.executable, "-m", "pip", "install", "-r", "requirements.txt"])
    import matplotlib.pyplot as plt
    import numpy as np
    import torch
    import torch.nn as nn
    from tqdm import trange
    import seaborn as sns
if int(np.__version__.split('.')[0]) >= 2:
    import subprocess
    import sys
    print("NumPy version >= 2 detected. Downgrading to a compatible version...")
    subprocess.check_call([sys.executable, "-m", "pip", "install", "numpy<2"])
    print("Please restart the kernel and rerun the script.")

np.random.seed(1)
torch.random.manual_seed(1)

plt.rcParams['font.size'] = 12
plt.rcParams['lines.markersize'] = 10
plt.rcParams['legend.fontsize'] = 10
colors = sns.color_palette([(0.6,0.8,.8), (1,0.7,0.3), (0.2,0.7,0.2), (0.8,0,0.2), (0,0.4,1), (0.6,0.5,.9), (0.5,0.3,.5)])

### Exact solution and data

Here, we will learn the dynamics of a 1D damped mass-spring system with unit mass, spring constant $k$ and damping coefficient $\mu$:
$$
\begin{aligned}
\dot{q} &= p, \\
\dot{p} &= -\mu p - k q.
\end{aligned}
$$

The system can be written as a pseudo-Hamiltonian system
$$
\begin{equation*}
    \begin{bmatrix}
 \dot{q}\\
 \dot{p}
\end{bmatrix} = \left(
\begin{bmatrix}
0 & 1\\
-1 & 0
\end{bmatrix}
+
\begin{bmatrix}
0 & 0\\
0 & -\mu
\end{bmatrix}
\right) 
\begin{bmatrix}
\frac{\partial \mathcal{H}}{\partial q}(q,p)\\
\frac{\partial \mathcal{H}}{\partial p}(q,p)
\end{bmatrix}
\end{equation*}
$$
with the Hamiltonian
$$
H(q, p) = \frac{1}{2}p^2 + \frac{k}{2}q^2.
$$


For the underdampened case, $\mu^2 < 4 k~,$ the exact solution is given by 
$$
\begin{align*}
q(t) &= e^{-\delta t}(A \cos(\phi + \omega t)),\\
p(t) &= -A e^{-\delta t}(\omega \sin(\phi + \omega t)) + \delta \cos(\phi + \omega t)
\end{align*}
$$
where $\omega=\frac{1}{2}\sqrt{4k - \mu^2}$ and the coefficients $A$ and $\phi$ determine the initial conditions. We will set $A=1$ and $\phi=0$ for the test data.

Let us turn the above exact solution into a function that we can use to generate a training data set $X_{\mathrm{train}}$ consisting of points $(t_i, x^j_i)$, where $t_i$ are evenly spaced times on some interval and $x^j_i=x^j(t_i)$ is given by the above exact solution for the initial state $x_0^j = x^j(t_0)$. We wil also create a test data set $X_{\mathrm{test}}$.

**Exercise:** Experiment with different values for the damping coefficient, including $\mu=0$, in which case the system is Hamiltonian.

In [ ]:
# ODE parameters:
MU = 0. # damping coefficient
K = 1 # spring constant

**Exercise:** Experiment with different number of training trajectories, number of data points and the end time of the training trajectories. These depend on the goal of the modelling: do you want to extrapolate only in time, or also interpolate in space. Also experiment with adding noise to your training data.

In [ ]:

# Data generation parameters
TRAJ_TRAIN = 5 # Number of distinct trajectories to train on
N_TRAIN = 5 # Number of data points in each training trajectory
TMAX_TRAIN = 1 # End time of each training trajectory
NOISE_STD = 0. # Standard deviation of the noise added to the data

TRAJ_TEST = 1 # Number of distinct trajectories to test on
N_TEST = 4*64 # Number of data points in each test trajectory
TMAX_TEST = 4 * (2*np.pi*np.sqrt(1/K)) # End time of each test trajectory

if TRAJ_TRAIN == 1:
    time_shift_max = 0
else:
    time_shift_max = 2*np.pi

def exact_solution(t, A=1, phi=0, k=K, mu=MU):
    """Get exact solution to the 1D underdamped harmonic oscillator."""
    assert mu**2 < 4 * k, "System must be underdamped."
    w = np.sqrt(4 * k - mu**2) / 2
    q = torch.exp(-mu / 2 * t) * A * torch.cos(phi + w * t)
    p = -A * torch.exp(-mu / 2 * t) * (w * torch.sin(phi + w * t) + mu / 2 * torch.cos(phi + w * t))
    x = torch.concat((q,p), axis = 1)
    return x

t_train = torch.linspace(0, TMAX_TRAIN, N_TRAIN + 1).unsqueeze(-1)
time_shifts = time_shift_max*torch.rand(1, TRAJ_TRAIN)
t_train_shifted = t_train.expand(-1, TRAJ_TRAIN).unsqueeze(1) + time_shifts.unsqueeze(0).repeat((t_train.shape[0],1,1))

t_test = torch.linspace(0, TMAX_TEST, N_TEST + 1).unsqueeze(-1)
time_shifts = time_shift_max*torch.rand(1, TRAJ_TEST)
t_test_shifted = t_test.expand(-1, TRAJ_TEST).unsqueeze(1) + time_shifts.unsqueeze(0).repeat((t_test.shape[0],1,1))

x_train = []
for i in range(TRAJ_TRAIN):
    if TRAJ_TRAIN == 1: # If we only have one training trajectory, we want to train and test on the same trajectory
        x_train.append(exact_solution(t_train_shifted[:,:,i]))
    else:
        A = torch.rand(1).item()+.3
        phi = torch.rand(1).item()-.5
        x_train.append(exact_solution(t_train_shifted[:,:,i], A, phi))
x_train = torch.stack(x_train, axis=2)
x_train = x_train + NOISE_STD * torch.randn_like(x_train)
x_test = exact_solution(t_test_shifted)

plt.figure(figsize=(12,3))
plt.plot(t_test, x_test[:, 0], color = 'k', linestyle="-", label="Test data (exact sol.)")
for i in range(min(TRAJ_TRAIN,len(colors))):
    plt.plot(t_train, x_train[:, 0, i], color = colors[i], linestyle="none", marker=".", label=f'Training trajectory {i+1}')
plt.title("Data")
plt.xlabel("$t$")
plt.ylabel("$q$")
plt.legend()
plt.show()

plt.figure(figsize=(5,5))
plt.plot(x_test[:, 0], x_test[:, 1], color = 'k', linestyle="-", label="Test data (exact sol.)")
for i in range(min(TRAJ_TRAIN,len(colors))):
    plt.plot(x_train[:, 0, i], x_train[:, 1, i], color = colors[i], linestyle="none", marker=".", label=f'Training trajectory {i+1}')
plt.xlabel("$q$")
plt.ylabel("$p$")
plt.show()

### Set up a standard feedforward neural network

**Optional exercise:** Experiment with different neural network architectures.

In [ ]:
class FNN(nn.Module):

    def __init__(self, input_size=1, hidden_size=32, output_size=1):
        super().__init__()
        
        self.fnn = nn.Sequential(
            nn.Linear(input_size, hidden_size),
            nn.Tanh(),
            nn.Linear(hidden_size, hidden_size),
            nn.ReLU(),
            nn.Linear(hidden_size, output_size),
        )

    def forward(self, x):
        return self.fnn(x)

### Set up the training process


We are not aiming to predict states of $q$ and $p$ directly, but rather learn a model $\hat{g}_\theta$ for the vector field $g : \mathbb{R}^2 \rightarrow \mathbb{R}^2$ so that the first-order ordinary differential equation (ODE)
$$
\dot{x} = g(x),
$$
where
$$
x = \begin{bmatrix}
q \\
p
\end{bmatrix},
$$
describes the system from which the data is obtained. Thus, we want to have a loss function
$$
\mathcal{L} = \lVert \dot{x} - \hat{g}_\theta(x) \rVert.
$$

Unfortunately, we do not have the time-derivates of $q$ and $p$ (we do not assume to know that $p = \dot{q}$), so these must be approximated. By approximating them with finite difference and evaluating $g$ on the midpoint between each data point, we effectively train on the implicit midpoint integration scheme
$$
\begin{equation*}
\frac{x_{i+1} - x_{i}}{\Delta t} = g \left( \frac{x_{i}+x_{i+1}}{2} \right),
\end{equation*}
$$
where $\Delta t = t_{n+1} - t_n$.

In [ ]:
x_midpoint = ((x_train + x_train.roll(shifts=1, dims=0))[1:,:,:]/2).requires_grad_()
dxdt = (x_train - x_train.roll(shifts=1, dims=0))[1:,:,:]/(t_train - t_train.roll(shifts=1, dims=0))[1:,:].reshape(-1,1,1)

In [ ]:
# Stacking the data for training:
x_midpoint_splits = torch.split(x_midpoint, 1, dim=2)
x_midpoint_reshaped = torch.stack(x_midpoint_splits, dim=0).squeeze(2).reshape(-1, 2).requires_grad_()
dxdt_splits = torch.split(dxdt, 1, dim=2)
dxdt_reshaped = torch.stack(dxdt_splits, dim=0).squeeze(2).reshape(-1, 2)

In [ ]:
def train(model, x, dxdt, nepochs=10000, learning_rate=1e-3):
    optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)
    losses = []
    
    with trange(nepochs, desc="Training the model") as pbar:
        for i in range(nepochs):
            optimizer.zero_grad()

            # Compute the loss from the difference between the left-hand side and right-hand side of the ODE:
            ode_rhs = model(x)
            loss = torch.mean((dxdt - ode_rhs) ** 2)
            
            # Backpropagation and optimization step:
            loss.backward(retain_graph=True)
            optimizer.step()
            
            # Log the loss value:
            losses.append(loss.item())
            if i % 100 == 0 or i == nepochs - 1:
                pbar.set_postfix(loss=loss.item())
            pbar.update(1)
    
    # Plot the loss curve
    plt.figure(figsize=(7, 4))
    plt.plot(losses)
    plt.yscale('log')
    plt.xlabel('Epoch')
    plt.ylabel('Loss')
    plt.show()

### Training a purely data-driven model

For comparison, we will first train a model that does not assume that the system has a Hamiltonian structure. We assume then only that the system can be formulated as a first-order ODE $\dot{x} = g(x)$.

**Exercise:** Set the input and output dimensions of the neural network:

In [ ]:
model_snn = FNN(SET_INPUT_DIM, 32, SET_OUTPUT_DIM)

In [ ]:
train(model_snn, x_midpoint_reshaped, dxdt_reshaped, nepochs=20000)

### Training an PHNN model

**Exercise:** Set the input and output dimensions of the PHNN.

**Exercise:** Experiment with making the damping coefficient not learnable, so that the model becomes a pure Hamiltonian neural network (HNN).

In [ ]:
class PHNN(nn.Module):
    def __init__(self, input_dim=SET_INPUT_DIM, hidden_dim=32):
        super().__init__()
        self.hamiltonian_net = FNN(input_size=input_dim, hidden_size=hidden_dim, output_size=SET_OUTPUT_DIM)
        self.mu = nn.Parameter(torch.tensor(0.0), requires_grad=False)
        self.S = torch.tensor([[0.0, 1.0], [-1.0, 0.0]])

    def hamiltonian(self, x):
        return self.hamiltonian_net(x)

    def grad(self, x):
        return torch.autograd.grad(self.hamiltonian(x).sum(), x, create_graph=True)[0]

    def forward(self, x):
        S = self.S.clone().detach()
        S[1, 1] = -self.mu
        dH = self.grad(x)
        return (S @ dH.T).T

In [ ]:
model_phnn = PHNN()
train(model_phnn, x_midpoint_reshaped, dxdt_reshaped, nepochs=20000)

In [ ]:
print(f"Learned damping coefficient: {model_phnn.mu.item():.3f}")

### Integrating the learned models:

We have learned two different models for the vector field $g$ that describes the dynamics of the systems by the ODE $\dot{x} = g{x}$. To obtain the predictions of $x$ we need to integrate this model from an inital state $x_0 = x(t_0)$. Let us use the Runge–Kutta method for this.

In [ ]:
def rk4(g, x0, t_end, dt):
    x = x0
    num_steps = int(t_end / dt)
    solution = torch.zeros(num_steps+1, *x0.shape)
    solution[0] = x0
    
    for i in range(1, num_steps+1):
        k1 = dt * g(x)
        k2 = dt * g(x + 0.5 * k1)
        k3 = dt * g(x + 0.5 * k2)
        k4 = dt * g(x + k3)
        
        x = x + (1/6) * (k1 + 2*k2 + 2*k3 + k4)
        solution[i] = x
    
    return solution

In [ ]:
x0 = x_test[0,:,0].requires_grad_()

x_test_snn = rk4(model_snn, x0, t_test[-1], TMAX_TEST/N_TEST).detach().numpy()
x_test_phnn = rk4(model_phnn, x0, t_test[-1], TMAX_TEST/N_TEST).detach().numpy()

In [ ]:
plt.figure(figsize=(5,5))
plt.plot(x_test[:, 0], x_test[:, 1], "k-", label="Exact solution")
plt.plot(x_test_snn[:, 0], x_test_snn[:, 1], "r-", label="Standard NN")
plt.plot(x_test_phnn[:, 0], x_test_phnn[:, 1], "g-", label="PHNN")
buffer_x = 0.1 * (x_test[:, 0].max() - x_test[:, 0].min())
buffer_y = 0.1 * (x_test[:, 1].max() - x_test[:, 1].min())
plt.xlim(x_test[:, 0].min() - buffer_x, x_test[:, 0].max() + buffer_x)
plt.ylim(x_test[:, 1].min() - buffer_y, x_test[:, 1].max() + buffer_y)
plt.xlabel("$q$")
plt.ylabel("$p$")
plt.legend(loc='upper left', bbox_to_anchor=(1, 1))
plt.show()

In [ ]:
plt.figure(figsize=(12,3))
plt.plot(t_test, x_test[:, 0], 'k-', linewidth=2, label='Exact solution')
plt.plot(t_test, x_test_snn[:, 0], 'r-', linewidth=2, label='Standard NN solution')
plt.plot(t_test, x_test_phnn[:, 0], 'g-', linewidth=2, label='PHNN solution')
plt.xlabel("$t$")
plt.ylabel("$q$")
plt.legend(fontsize=12, loc='upper center', bbox_to_anchor=(0.5, -0.25), ncol=3)
plt.show()

In [ ]:
# Compute the mean squared errors:
mse_snn = torch.mean((x_test[:,:,0] - x_test_snn)**2, axis=1)
mse_phnn = torch.mean((x_test[:,:,0] - x_test_phnn)**2, axis=1)

plt.figure(figsize=(12,3))
plt.semilogy(t_test, mse_snn, "r-", label="Standard NN")
plt.semilogy(t_test, mse_phnn, "g-", label="PHNN")
plt.xlabel("$t$")
plt.ylabel("MSE")
plt.legend()
plt.show()

In [ ]:
# Compute Hamiltonians:
H_exact = 0.5 * (x_test[:, 1].numpy()**2 + K * x_test[:, 0].numpy()**2)
H_phnn = 0.5 * (x_test_phnn[:, 1]**2 + K * x_test_phnn[:, 0]**2)
modelH_exact = model_phnn.hamiltonian(x_test[:,:,0]).detach().numpy()
modelH_phnn = model_phnn.hamiltonian(torch.from_numpy(x_test_phnn).float()).detach().numpy()

plt.figure(figsize=(12, 3))
plt.plot(t_test, H_exact, 'k-', linewidth=2, label='Exact Hamiltonian on true data')
plt.plot(t_test, H_phnn, 'g-', linewidth=2, label='Exact Hamiltonian on PHNN prediction')
plt.plot(t_test, modelH_exact, 'k--', linewidth=2, label='Learned Hamiltonian on true data')
plt.plot(t_test, modelH_phnn, 'g--', linewidth=2, label='Learned Hamiltonian on PHNN prediction')
plt.xlabel('Time')
plt.ylabel('Hamiltonian')
plt.legend(fontsize=12, loc='upper center', bbox_to_anchor=(0.5, -0.1), ncol=2)
plt.show()


### Further modelling

The [phlearn](https://github.com/SINTEF/pseudo-hamiltonian-neural-networks) package can be used to set up PHNNs that can learn systems of arbitrary dimensions and with external forces. As a first step, [this notebook](https://github.com/SINTEF/pseudo-hamiltonian-neural-networks/blob/main/example_scripts/spring_example.ipynb) shows how to train a PHNN model for a damped mass-spring system with external forces acting on it. That is,
$$
\begin{aligned}
\frac{d q}{d t} &= p, \\
\frac{d p}{d t} &= -\mu p - k q + f(q, p, t).
\end{aligned}
$$